<a href="https://colab.research.google.com/github/imrohu/NLP/blob/master/Assignment2NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
from google.colab import drive
drive.mount('/content/gdrive')
import os



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [44]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import keras 
import random
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re 
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

url='https://raw.githubusercontent.com/cacoderquan/Sentiment-Analysis-on-the-Rotten-Tomatoes-movie-review-dataset/master/train.tsv'
data=pd.read_csv(url, sep='\t', header=0, quoting=3)

data.head(100)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1
1,2,1,A series of escapades demonstrating the adage that what is good for the goose,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
...,...,...,...,...
95,96,3,", I suspect ,",2
96,97,3,"I suspect ,",2
97,98,3,I suspect,2
98,99,3,I,2


In [45]:
data.shape

(156060, 4)

In [46]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re 

pd.set_option('max_colwidth',400)

wordnet = WordNetLemmatizer()
stopwords_en = stopwords.words("english")
punctuations = "?:!.,;-()"

raw_reviews = data.Phrase.values
cleaned_reviews = []

for i in range(len(raw_reviews)):
  review = str(raw_reviews[i])
  review=re.sub('[^a-zA-Z]',' ',review)
  review=[wordnet.lemmatize(w) for w in word_tokenize(str(review).lower())]
  review=' '.join(review)
  cleaned_reviews.append(review)

data['cleaned_reviews'] = cleaned_reviews

data.head()

,PhraseId,SentenceId,Phrase,Sentiment,cleaned_reviews
0,1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1,a series of escapade demonstrating the adage that what is good for the goose is also good for the gander some of which occasionally amuses but none of which amount to much of a story
1,2,1,A series of escapades demonstrating the adage that what is good for the goose,2,a series of escapade demonstrating the adage that what is good for the goose
2,3,1,A series,2,a series
3,4,1,A,2,a
4,5,1,series,2,series


In [0]:
from keras.utils import to_categorical
X = data.cleaned_reviews.values
Y = to_categorical(data.Sentiment.values)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = TfidfVectorizer(stop_words = None, ngram_range=(1,2))
#X = vectorizer.fit_transform(X).toarray()
data1 = data.cleaned_reviews.values

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.3, random_state=2003)

In [49]:
from nltk import FreqDist
all_words=' '.join(x_train)
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)

num_unique_word=len(dist)
num_unique_word

13725

In [50]:
r_len=[]
for text in x_train:
    word=word_tokenize(text)
    l=len(word)

    r_len.append(l)
    
MAX_REVIEW_LEN=np.max(r_len)
MAX_REVIEW_LEN

48

In [0]:
max_features = num_unique_word
max_words = MAX_REVIEW_LEN
batch_size = 128
epochs = 10
num_classes=5

In [0]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(x_train)
x_train = tokenizer.texts_to_sequences(x_train)
x_val = tokenizer.texts_to_sequences(x_val)

x_test = tokenizer.texts_to_sequences(data1)

In [53]:
from keras.preprocessing import sequence,text
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
x_train = sequence.pad_sequences(x_train, maxlen=max_words)
x_val = sequence.pad_sequences(x_val, maxlen=max_words)
x_test = sequence.pad_sequences(x_test, maxlen=max_words)
#print(X_train.shape,X_val.shape)
x_test

array([[   0,    0,    0, ...,    3,    2,   39],
       [   0,    0,    0, ...,   14,    1, 3134],
       [   0,    0,    0, ...,    0,    2,  338],
       ...,
       [   0,    0,    0, ...,    0, 8655, 7961],
       [   0,    0,    0, ...,    0,    0, 8655],
       [   0,    0,    0, ...,    0,    0, 7961]], dtype=int32)

In [0]:
from keras.layers import Input, Dense, Embedding, Flatten
from keras.layers import SpatialDropout1D
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import Sequential

In [0]:
model2 = Sequential()

# Input / Embdedding
model2.add(Embedding(max_features, 200, input_length=max_words))

# CNN
model2.add(SpatialDropout1D(0.2))

model2.add(Conv1D(64, kernel_size=3, padding='same', activation='relu'))
model2.add(MaxPooling1D(pool_size=2))

model2.add(Conv1D(128, kernel_size=3, padding='same', activation='relu'))
model2.add(MaxPooling1D(pool_size=2))

model2.add(Flatten())

# Output layer
model2.add(Dense(4, activation='softmax'))

In [0]:
from keras import backend as Ke
def recall_nlp(true, pred):
    t_p = Ke.sum(Ke.round(Ke.clip(true * pred, 0, 1)))
    p_p = Ke.sum(Ke.round(Ke.clip(true, 0, 1)))
    recall = t_p/ (p_p + Ke.epsilon())
    return recall

def precision_nlp(true, pred):
    t_p = Ke.sum(Ke.round(Ke.clip(true * pred, 0, 1)))
    p_p = Ke.sum(Ke.round(Ke.clip(pred, 0, 1)))
    precision = t_p / (p_p + Ke.epsilon())
    return precision

def f1_nlp(true, pred):
    precision = precision_nlp(true, pred)
    recall = recall_nlp(true, pred)
    return 2*((precision*recall)/(precision+recall+Ke.epsilon()))

In [57]:
from keras import metrics
from sklearn.metrics import f1_score
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[metrics.categorical_accuracy,f1_nlp,precision_nlp, recall_nlp])
model2.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=epochs, batch_size=batch_size, verbose=1)

Train on 109242 samples, validate on 46818 samples
Epoch 1/10
109242/109242 [==============================] - 11s 96us/step - loss: 1.0165 - categorical_accuracy: 0.5915 - f1_nlp: 0.5410 - precision_nlp: 0.6652 - recall_nlp: 0.4626 - val_loss: 0.8624 - val_categorical_accuracy: 0.6459 - val_f1_nlp: 0.6169 - val_precision_nlp: 0.6906 - val_recall_nlp: 0.5578
Epoch 2/10
109242/109242 [==============================] - 10s 89us/step - loss: 0.7788 - categorical_accuracy: 0.6834 - f1_nlp: 0.6653 - precision_nlp: 0.7201 - recall_nlp: 0.6187 - val_loss: 0.8330 - val_categorical_accuracy: 0.6569 - val_f1_nlp: 0.6315 - val_precision_nlp: 0.6982 - val_recall_nlp: 0.5768
Epoch 3/10
109242/109242 [==============================] - 10s 89us/step - loss: 0.6867 - categorical_accuracy: 0.7168 - f1_nlp: 0.7061 - precision_nlp: 0.7468 - recall_nlp: 0.6699 - val_loss: 0.8419 - val_categorical_accuracy: 0.6602 - val_f1_nlp: 0.6462 - val_precision_nlp: 0.6855 - val_recall_nlp: 0.6115
Epoch 4/10
109242/1

In [58]:
loss, accuracy, f1_score, precision, recall = model2.evaluate(x_val, y_val, verbose=0)
print("Accuracy " + str(accuracy) + ":\n\ff1_score = " + str(f1_score) +
          "\nPrecision = " + str(precision) + "\nRecall = " + str(recall))

Accuracy 0.6403092827544962:
f1_score = 0.6355485329665064
Precision = 0.6508624560708008
Recall = 0.6214276560297322


In [0]:
model2.save("1107345.pt")